# solving PDE through stochastic simulation  
* a reference can be found at [exit problem for PDE](https://math.nist.gov/mcsd/Seminars/2006/2006-10-18-mascagni-presentation.pdf)

for general elliptic PDE:  
$$
Lu(x) - c(x)u(x) = g(x),\quad x\in \Omega
\\
u(x) = f(x), \quad x\in \partial\Omega
$$
where the operator
$$
L = b_i\partial_i + \frac{1}{2}a_{ij}\partial_i\partial_j
\\
a_{ij} = \sigma_{ik}(\sigma^T)_{kj} = \sigma_{ik}\sigma_{jk}
$$
the wiener integral path gives:  
$$
u(x) = \mathbb{E}^L_x \big[ \int_0^{t_{\partial\Omega}} \{\frac{f(X^x(t_{\partial\Omega}))}{t_{\partial\Omega}} - g(X^x(t)) \}e^{-\int_0^t c(X^x(s))ds} dt \big]
$$  
with the stochastic path:  
$$
dX^x (t) = b(X^x,t) dt + \sigma(X^x, t) dW
\\
X^x(0) = x
$$

for problem:  
$$
x\partial_x u + y\partial_y u + \frac{1}{2}\Delta u = x^2+y^2+1
\\
u(\partial \Omega) = 1/2, \quad \partial\Omega : x^2+y^2=1
$$
the exact solution is 
$$
u(x,y) = \frac{1}{2}(x^2+y^2)
$$
so
$$
u(x,y) = \mathbb{E}_{x,y}^L [ f(X^{x,y}(t_{\partial\Omega})) - \int_0^{t_{\partial\Omega}}g(X)dt]
$$

In [53]:
import numpy as np

def check_if_exit(v):
    # Takes a vector v=(x,y)
    # Checks if v has intersected with the boundary of D
    return (np.linalg.norm(v,2) >=1)

def em(x,dt):
    # b = (x,y), sigma = 1
    x += x * dt + np.random.normal(size=2,scale=np.sqrt(dt))

g = lambda x : 1+x[0]*x[0]+x[1]*x[1]

def simulate_exit_time(v):
    # Simulates exit time starting at v=(x,y), returns exit position
    delta_t = 0.0005
    exit = False

    # Copy because simulation modifies in place
    if hasattr(v,'copy'): # For NumPy arrays
        x = v.copy() 
    else:
        x = np.array(v) # We input a non-NumPy array
    
    int_g = 0
    while not check_if_exit(x):
        em(x, delta_t)
        int_g += g(x) * delta_t
    return int_g

r2 = np.sqrt(0.5)
v=np.array((0.,0.5)) # The origin
print(v)

u = lambda x : 0.5*(x[0]**2+x[1]**2)
f = lambda x : np.cos(np.arctan2(x[1],x[0]))

def get_exp_f_exit(starting_point, n_trials): 
    return np.mean([ 0.5 - simulate_exit_time(v) for k in range(0,n_trials)])

exp_f_exit = get_exp_f_exit(v,2000) # Expected value of f(Exit(x,d))
print('The value u(v) = %s\nThe value of Exp(f(Exit))=%s' %(u(v), exp_f_exit)) 



[ 0.   0.5]
The value u(v) = 0.125
The value of Exp(f(Exit))=0.106156363066
